In [1]:
!sudo apt-get install swi-prolog
# !pip install pyswip
!pip install swiplserver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  autopoint debhelper debugedit dh-autoreconf dh-strip-nondeterminism dwz
  gettext gettext-base intltool-debian libarchive-cpio-perl
  libarchive-zip-perl libdebhelper-perl libfile-stripnondeterminism-perl
  libmail-sendmail-perl libossp-uuid16 libsub-override-perl
  libsys-hostname-long-perl libtool po-debconf swi-prolog-core
  swi-prolog-core-packages swi-prolog-doc swi-prolog-nox swi-prolog-x
Suggested packages:
  dh-make gettext-doc libasprintf-dev libgettextpo-dev uuid libtool-doc
  gcj-jdk libmail-box-perl elpa-ediprolog swi-prolog-java swi-prolog-odbc
  swi-prolog-bdb
The following NEW packages will be installed:
  autopoint debhelper debugedit dh-autoreconf dh-strip-nondeterminism dwz
  gettext gettext-base intltool-debian libarchive-cpio-perl
  libarchive-zip-perl libdebhelper-perl libfile-stripnondeterminism-perl
  libmail-send

# Prolog <knowledge_base.pl>

In [ ]:
# @title
# CHECKPOINT Prolog Predicate File
%%writefile knowledge_base.pl
:- use_module(library(http/json)).

read_json_file(FileName, JSON) :-
    open(FileName, read, Stream),
    json_read_dict(Stream, JSON),
    close(Stream).

write_json_file(FileName, JSON) :-
    open(FileName, write, Stream),
    json_write_dict(Stream, JSON),
    close(Stream).

% Trigger update of all cells
update_percepts :-
    FileName = 'game_state.json',
    read_json_file(FileName, JSON),
    update_all_percepts(JSON, UpdatedJSON),
    write_json_file(FileName, UpdatedJSON).

% Apply updates to each cell
update_all_percepts(JSON, UpdatedJSON) :-
    maplist(update_cell_percepts(JSON), JSON, UpdatedJSON).

% Update percepts for a single cell
update_cell_percepts(JSON, Cell, UpdatedCell) :-
    update_stenchy(JSON, Cell, CellWithStench),
    update_windy(JSON, CellWithStench, CellWithWind),
    update_glitter(JSON, CellWithWind, UpdatedCell).

% Update 'stenchy' based on presence of 'wumpus' in adjacent cells
update_stenchy(JSON, Cell, UpdatedCell) :-
    (   adjacent_has_feature(JSON, Cell, wumpus)
    ->  get_dict(percepts, Cell, Percepts),
        put_dict(stenchy, Percepts, true, UpdatedPercepts),
        put_dict(percepts, Cell, UpdatedPercepts, UpdatedCell)
    ;   UpdatedCell = Cell).

% Update 'windy' based on presence of 'pit' in adjacent cells
update_windy(JSON, Cell, UpdatedCell) :-
    (   adjacent_has_feature(JSON, Cell, pit)
    ->  get_dict(percepts, Cell, Percepts),
        put_dict(windy, Percepts, true, UpdatedPercepts),
        put_dict(percepts, Cell, UpdatedPercepts, UpdatedCell)
    ;   UpdatedCell = Cell).

% Update 'glitter' based on presence of 'gold' in the same cell
update_glitter(JSON, Cell, UpdatedCell) :-
    (   cell_has_feature(Cell, gold)
    ->  get_dict(percepts, Cell, Percepts),
        put_dict(glitter, Percepts, true, UpdatedPercepts),
        put_dict(percepts, Cell, UpdatedPercepts, UpdatedCell)
    ;   UpdatedCell = Cell).

% Check for feature in any adjacent cell
adjacent_has_feature(JSON, Cell, Feature) :-
    get_dict(location, Cell, Loc),
    get_dict(x, Loc, X),
    get_dict(y, Loc, Y),
    (   adjacent_cell_feature(X, Y, 0, 1, JSON, Feature);  % Check up
        adjacent_cell_feature(X, Y, 0, -1, JSON, Feature); % Check down
        adjacent_cell_feature(X, Y, 1, 0, JSON, Feature);  % Check right
        adjacent_cell_feature(X, Y, -1, 0, JSON, Feature)). % Check left

% Helper to check specific adjacent cell for a feature
adjacent_cell_feature(X, Y, Dx, Dy, JSON, Feature) :-
    AdjX is X + Dx,
    AdjY is Y + Dy,
    member(Cell, JSON),
    get_dict(location, Cell, Loc),
    get_dict(x, Loc, AdjX),
    get_dict(y, Loc, AdjY),
    get_dict(probabilities, Cell, Prob),
    get_dict(Feature, Prob, FeatureValue),
    FeatureValue > 0.05.

% Check if current cell has a specific feature
cell_has_feature(Cell, Feature) :-
    get_dict(probabilities, Cell, Prob),
    get_dict(Feature, Prob, FeatureValue),
    FeatureValue > 0.05.

% Check if two cells are adjacent based on their coordinates
is_adjacent(Cell1, Cell2) :-
    X1 = Cell1.location.x, Y1 = Cell1.location.y,
    X2 = Cell2.location.x, Y2 = Cell2.location.y,
    (   X1 == X2, abs(Y1 - Y2) == 1
    ;   Y1 == Y2, abs(X1 - X2) == 1).

% Run the percept update process
:- update_percepts.

In [12]:
# Prolog Predicate File
%%writefile knowledge_base.pl
:- use_module(library(http/json)).

read_json_file(FileName, JSON) :-
    open(FileName, read, Stream),
    json_read_dict(Stream, JSON),
    close(Stream).

write_json_file(FileName, JSON) :-
    open(FileName, write, Stream),
    json_write_dict(Stream, JSON),
    close(Stream).

% Trigger update of all cells
update_percepts(_,_,_,_) :-
    FileName = 'game_state.json',
    read_json_file(FileName, JSON),
    update_all_percepts(JSON, UpdatedJSON),
    write_json_file(FileName, UpdatedJSON).

% Apply updates to each cell
update_all_percepts(JSON, UpdatedJSON) :-
    maplist(update_cell_percepts(JSON), JSON, UpdatedJSON).

% Update percepts for a single cell
update_cell_percepts(JSON, Cell, UpdatedCell) :-
    update_stenchy(JSON, Cell, CellWithStench),
    update_windy(JSON, CellWithStench, CellWithWind),
    update_glitter(JSON, CellWithWind, UpdatedCell).

% Update 'stenchy' based on presence of 'wumpus' in adjacent cells
update_stenchy(JSON, Cell, UpdatedCell) :-
    (   adjacent_has_feature(JSON, Cell, wumpus)
    ->  get_dict(percepts, Cell, Percepts),
        put_dict(stenchy, Percepts, true, UpdatedPercepts),
        put_dict(percepts, Cell, UpdatedPercepts, UpdatedCell)
    ;   UpdatedCell = Cell).

% Update 'windy' based on presence of 'pit' in adjacent cells
update_windy(JSON, Cell, UpdatedCell) :-
    (   adjacent_has_feature(JSON, Cell, pit)
    ->  get_dict(percepts, Cell, Percepts),
        put_dict(windy, Percepts, true, UpdatedPercepts),
        put_dict(percepts, Cell, UpdatedPercepts, UpdatedCell)
    ;   UpdatedCell = Cell).

% Update 'glitter' based on presence of 'gold' in the same cell
update_glitter(_, Cell, UpdatedCell) :-
    (   cell_has_feature(Cell, gold)
    ->  get_dict(percepts, Cell, Percepts),
        put_dict(glitter, Percepts, true, UpdatedPercepts),
        put_dict(percepts, Cell, UpdatedPercepts, UpdatedCell)
    ;   UpdatedCell = Cell).

% Check for feature in any adjacent cell
adjacent_has_feature(JSON, Cell, Feature) :-
    get_dict(location, Cell, Loc),
    get_dict(x, Loc, X),
    get_dict(y, Loc, Y),
    (   adjacent_cell_feature(X, Y, 0, 1, JSON, Feature);  % Check up
        adjacent_cell_feature(X, Y, 0, -1, JSON, Feature); % Check down
        adjacent_cell_feature(X, Y, 1, 0, JSON, Feature);  % Check right
        adjacent_cell_feature(X, Y, -1, 0, JSON, Feature)). % Check left

% Helper to check specific adjacent cell for a feature
adjacent_cell_feature(X, Y, Dx, Dy, JSON, Feature) :-
    AdjX is X + Dx,
    AdjY is Y + Dy,
    member(Cell, JSON),
    get_dict(location, Cell, Loc),
    get_dict(x, Loc, AdjX),
    get_dict(y, Loc, AdjY),
    get_dict(probabilities, Cell, Prob),
    get_dict(Feature, Prob, FeatureValue),
    FeatureValue > 0.05.

% Check if current cell has a specific feature
cell_has_feature(Cell, Feature) :-
    get_dict(probabilities, Cell, Prob),
    get_dict(Feature, Prob, FeatureValue),
    FeatureValue > 0.05.

% Check if two cells are adjacent based on their coordinates
is_adjacent(Cell1, Cell2) :-
    X1 = Cell1.location.x, Y1 = Cell1.location.y,
    X2 = Cell2.location.x, Y2 = Cell2.location.y,
    (   X1 == X2, abs(Y1 - Y2) == 1
    ;   Y1 == Y2, abs(X1 - X2) == 1).

% Run the percept update process
:- update_percepts(_,_,_,_).

Overwriting knowledge_base.pl


In [ ]:
!cat knowledge_base.pl

# Python File

In [21]:
import json
import sys
import atexit
from swiplserver import PrologMQI, PrologThread
import logging

prolog_file = "knowledge_base.pl"

logging.basicConfig(filename='app.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s')

def initialize_game_state(filename):
    """ Initializes the game state with the specified initial probabilities for each cell, excluding the start cell (1,1). """
    state = []
    for i in range(1, 5):
        for j in range(1, 5):
            if i == 1 and j == 1:
                # Exclude the starting position from having any Wumpus, pits, or gold
                probs = {'pit': 0, 'wumpus': 0, 'gold': 0}
            else:
                probs = {'pit': 3/15, 'wumpus': 1/15, 'gold': 1/15}
            state.append({
                'location': {'x': i, 'y': j},
                'percepts': {'stenchy': False, 'glitter': False, 'windy': False},
                'probabilities': probs,
                'visited': False
            })
    with open(filename, 'w') as file:
        json.dump(state, file, indent=4)

def update_game_state_from_prolog(prolog_file, filename):
    """ Uses Prolog to update the percepts and stores updates in a JSON file. """
    results = []
    try:
      with PrologMQI() as mqi:
        with mqi.create_thread() as prolog_thread:
              prolog_thread.query("consult('knowledge_base.pl')")
              results = prolog_thread.query("update_percepts(X, Y, Percept, State)")
    except Exception as e:
      print("An error occurred during update_game_state_from_prolog:", str(e))

    with open(filename, 'r+') as file:
        state = json.load(file)
        for result in results:
            x, y, percept, percept_state = result['X'], result['Y'], result['Percept'], result['State']
            for cell in state:
                if cell['location']['x'] == x and cell['location']['y'] == y:
                    cell['percepts'][percept] = percept_state
        file.seek(0)
        json.dump(state, file, indent=4)

def calculate_joint_probabilities(filename):
    """ Calculates and updates the joint probability of each cell being safe, based on Bayesian updating from sensory data. """
    with open(filename, 'r') as file:
        state = json.load(file)

    for cell in state:
        x, y = cell['location']['x'], cell['location']['y']
        pit_prob = cell['probabilities']['pit']
        wumpus_prob = cell['probabilities']['wumpus']

        p_stench = p_breeze = 1
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            adj_x, adj_y = x + dx, y + dy
            adjacent_cell = next((item for item in state if item['location']['x'] == adj_x and item['location']['y'] == adj_y), None)
            if adjacent_cell:
                if adjacent_cell['percepts']['stenchy']:
                    p_stench *= 0.75  # Likelihood of stench given Wumpus
                else:
                    p_stench *= 0.25  # Complement of the likelihood
                if adjacent_cell['percepts']['windy']:
                    p_breeze *= 0.75  # Likelihood of breeze given Pit
                else:
                    p_breeze *= 0.25  # Complement of the likelihood

        updated_wumpus_prob = (p_stench * wumpus_prob) / (p_stench * wumpus_prob + (1 - p_stench) * (1 - wumpus_prob))
        updated_pit_prob = (p_breeze * pit_prob) / (p_breeze * pit_prob + (1 - p_breeze) * (1 - pit_prob))

        # Update probabilities in the state
        cell['probabilities']['wumpus'] = updated_wumpus_prob
        cell['probabilities']['pit'] = updated_pit_prob

        # Calculate safety probability
        safe_prob = (1 - updated_pit_prob) * (1 - updated_wumpus_prob)
        print(f"Cell {(x, y)} has a safety probability of {safe_prob:.2f}")

    # Write updated probabilities back to file
    with open(filename, 'w') as file:
        json.dump(state, file, indent=4)


def main():
    json_file = 'game_state.json'
    print("\n---------------------------------------------\n")
    print("    Welcome to Group 4's Wumpus World!")
    print("\n---------------------------------------------\n")

    # Initialize game state
    try:
        initialize_game_state(json_file)
        logging.info("Game state initialized successfully.")
    except Exception as e:
        logging.error("Error initializing game state: %s", e)

    # Update game state from Prolog
    try:
        update_game_state_from_prolog(prolog_file, json_file)
        logging.info("Game state updated from Prolog successfully.")
    except Exception as e:
        logging.error("Error updating game state from Prolog: %s", e)

    # Calculate and display joint probabilities
    try:
        calculate_joint_probabilities(json_file)
        logging.info("Joint probabilities calculated successfully.")
    except Exception as e:
        logging.error("Error calculating joint probabilities: %s", e)

if __name__ == "__main__":
    main()


---------------------------------------------

    Welcome to Group 4's Wumpus World!

---------------------------------------------

Cell (1, 1) has a safety probability of 1.00
Cell (1, 2) has a safety probability of 0.80
Cell (1, 3) has a safety probability of 0.80
Cell (1, 4) has a safety probability of 0.69
Cell (2, 1) has a safety probability of 0.80
Cell (2, 2) has a safety probability of 0.87
Cell (2, 3) has a safety probability of 0.87
Cell (2, 4) has a safety probability of 0.80
Cell (3, 1) has a safety probability of 0.80
Cell (3, 2) has a safety probability of 0.87
Cell (3, 3) has a safety probability of 0.87
Cell (3, 4) has a safety probability of 0.80
Cell (4, 1) has a safety probability of 0.69
Cell (4, 2) has a safety probability of 0.80
Cell (4, 3) has a safety probability of 0.80
Cell (4, 4) has a safety probability of 0.69
